In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,exact,family,trouble,from,humour,country,speaker,solicit,crimson,conquest,...,command,nest,even,closer,respectful,dinah,sage,fountain,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.989028213166

Test set score: 0.886748120301


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.957993730408

Test set score: 0.915883458647


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.886833855799

Test set score: 0.874060150376


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.697613721104


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0:

Let's try some of those things. Let's add some features:
- percentage of characters that are punctuation
- ratio of nouns/verbs/adj to sentence length
- sentence length
- ratio of stop words to sentence length

In [93]:
df = word_counts.copy()

In [17]:
df.head()

,exact,family,trouble,from,humour,country,speaker,solicit,crimson,conquest,...,command,nest,even,closer,respectful,dinah,sage,fountain,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


I'm going to update the following cells as I go, and record results here instead of making new cells every time I run the model.

Added punct_ratio, noun_ratio, verb_ratio, sents_len, stop_ratio.
- Training set score: 0.960501567398
- Test set score: 0.919642857143
    
Added word_len.
- Training set score: 0.960188087774
- Test set score: 0.918233082707

Ran GridSearchCV -- already using optimal C value.

In [47]:
df['punct_ratio'] = 0
df['noun_ratio'] = 0
df['verb_ratio'] = 0
df['adj_ratio'] = 0
df['sents_len'] = 0
df['stop_ratio'] = 0
df['word_len'] = 0

for i, sentence in enumerate(df['text_sentence']):
    punct_count, noun_count, verb_count, adj_count, stop_count, char_count = 0, 0, 0, 0, 0, 0
    for word in df.loc[i, 'text_sentence']:
        punct_count += word.is_punct
        noun_count += (word.pos_=='NOUN')
        verb_count += (word.pos_=='VERB')
        adj_count += (word.pos_=='ADJ')
        stop_count += word.is_stop
    df.loc[i, 'punct_ratio'] = punct_count/len(sentence)
    df.loc[i, 'noun_ratio'] = noun_count/len(sentence)
    df.loc[i, 'verb_ratio'] = verb_count/len(sentence)
    df.loc[i, 'adj_ratio'] = adj_count/len(sentence)
    df.loc[i, 'stop_ratio'] = stop_count/len(sentence)
    df.loc[i, 'sents_len'] = len(sentence)
    df.loc[i, 'word_len'] = len(str(sentence))/len(sentence)
    
    if i % 500 == 0:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [132]:
df.head()

,exact,family,trouble,from,humour,country,speaker,solicit,crimson,conquest,...,ADJ,ADP,NOUN,DET,PUNCT,CCONJ,PRON,INTJ,NUM,X
0,0,0,0,0,0,0,0,0,0,0,...,0.044776,0.119403,0.179104,0.074627,0.149254,0.089552,0.044776,0.000000,0.0,0.0
1,0,0,1,0,0,0,0,0,0,0,...,0.111111,0.126984,0.126984,0.095238,0.111111,0.031746,0.063492,0.000000,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.030303,0.121212,0.060606,0.090909,0.121212,0.030303,0.060606,0.060606,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.666667,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.166667,0.000000,0.000000,0.000000,0.333333,0.000000,0.166667,0.000000,0.0,0.0


In [49]:
Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [50]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3069) (3190,)
Training set score: 0.960188087774

Test set score: 0.918233082707


In [56]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [55]:
svc = SVC()
svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

Training set score: 0.682445141066

Test set score: 0.691729323308


Hmm... Well, SVM didn't do very well...

In [62]:
lr_params = {'C': [0.0001, 0.01, 1, 100, 10000]}
lr_grid = GridSearchCV(lr, lr_params, cv=4)
lr_grid.fit(X_train, y_train)
print(lr_grid.best_params_)
print(lr_grid.best_score_)

{'C': 1}
0.90407523511


Okay, so we'll stick with C=1. Let's try using all the pos\_ values as columns.

In [120]:
df = word_counts.copy()

df['sents_len'] = 0
df['punct_ratio'] = 0
df['stop_ratio'] = 0
df['word_len'] = 0

for i, sentence in enumerate(df['text_sentence']):
    punct_count, stop_count = 0, 0
    for word in df.loc[i, 'text_sentence']:
        #If a column doesn't exist for the pos_, create it.
        if df.columns.isin([word.pos_]).sum()==0:
            df[word.pos_] = 0
        df.loc[i, word.pos_] += 1
        punct_count += word.is_punct
        stop_count += word.is_stop
    df.loc[i, 'sents_len'] = len(sentence)
    df.loc[i, 'punct_ratio'] = punct_count/len(sentence)
    df.loc[i, 'stop_ratio'] = stop_count/len(sentence)
    df.loc[i, 'word_len'] = len(str(sentence))/len(sentence)
    df.iloc[i, 3068:] /= len(sentence)
    
    if i % 500 == 0:
        print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [133]:
df.head()

,exact,family,trouble,from,humour,country,speaker,solicit,crimson,conquest,...,ADJ,ADP,NOUN,DET,PUNCT,CCONJ,PRON,INTJ,NUM,X
0,0,0,0,0,0,0,0,0,0,0,...,0.044776,0.119403,0.179104,0.074627,0.149254,0.089552,0.044776,0.000000,0.0,0.0
1,0,0,1,0,0,0,0,0,0,0,...,0.111111,0.126984,0.126984,0.095238,0.111111,0.031746,0.063492,0.000000,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.030303,0.121212,0.060606,0.090909,0.121212,0.030303,0.060606,0.060606,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.666667,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.166667,0.000000,0.000000,0.000000,0.333333,0.000000,0.166667,0.000000,0.0,0.0


In [122]:
Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [125]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.960188087774

Test set score: 0.918233082707


Not getting any better.....

# Challenge 1

Let's move on for the moment to Challenge 1, comparing other authors to these two. Let's try Shakespeare (which should be pretty different) and Edgewood (I don't know it...)

In [142]:
# Load and clean the data.
hamlet = gutenberg.raw('shakespeare-hamlet.txt')
parents = gutenberg.raw('edgeworth-parents.txt')

hamlet = text_cleaner(hamlet)
parents = text_cleaner(parents)

print(hamlet[:100])
print(parents[:100])

Actus Primus. Scoena Prima. Enter Barnardo and Francisco two Centinels. Barnardo. Who's there? Fran.
THE ORPHANS. Near the ruins of the castle of Rossmore, in Ireland, is a small cabin, in which there 


In [143]:
hamlet_doc = nlp(hamlet)
parents_doc = nlp(parents)

In [151]:
hamlet_sents = [[sent, 'Shakespeare'] for sent in hamlet_doc.sents]
parents_sents = [[sent, 'Edgewood'] for sent in parents_doc.sents]
hamlet_sents = hamlet_sents[:len(alice_sents)]
parents_sents = parents_sents[:len(alice_sents)]

In [160]:
ham_ali_sentences = pd.DataFrame(alice_sents + hamlet_sents)
ham_per_sentences = pd.DataFrame(hamlet_sents + persuasion_sents)
par_ali_sentences = pd.DataFrame(alice_sents + parents_sents)
par_per_sentences = pd.DataFrame(parents_sents + persuasion_sents)

#We'll keep the same common word features so our model will run.
ham_ali_bow = bow_features(ham_ali_sentences, common_words)
ham_per_bow = bow_features(ham_per_sentences, common_words)
par_ali_bow = bow_features(par_ali_sentences, common_words)
par_per_bow = bow_features(par_per_sentences, common_words)

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


In [163]:
def add_features(df):

    df['sents_len'] = 0
    df['punct_ratio'] = 0
    df['stop_ratio'] = 0
    df['word_len'] = 0

    for i, sentence in enumerate(df['text_sentence']):
        punct_count, stop_count = 0, 0
        for word in df.loc[i, 'text_sentence']:
            #If a column doesn't exist for the pos_, create it.
            if df.columns.isin([word.pos_]).sum()==0:
                df[word.pos_] = 0
            df.loc[i, word.pos_] += 1
            punct_count += word.is_punct
            stop_count += word.is_stop
        df.loc[i, 'sents_len'] = len(sentence)
        df.loc[i, 'punct_ratio'] = punct_count/len(sentence)
        df.loc[i, 'stop_ratio'] = stop_count/len(sentence)
        df.loc[i, 'word_len'] = len(str(sentence))/len(sentence)
        df.iloc[i, 3068:] /= len(sentence)
    
        if i % 500 == 0:
            print(i)
        
    return df

In [166]:
df_ham_ali = add_features(ham_ali_bow)

0
500
1000
1500
2000
2500
3000


In [167]:
df_ham_per = add_features(ham_per_bow)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [168]:
df_par_ali = add_features(par_ali_bow)

0
500
1000
1500
2000
2500
3000


In [180]:
df_par_per = add_features(par_per_bow)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000


In [187]:
# So I don't have to change all the previous code...
df = df_ham_ali.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

y_pred = lr.predict(X_test)
print(lr.score(X_test, y_test))
print(pd.crosstab(y_test, y_pred))

0.438622754491
col_0        Austen  Carroll
text_source                 
Carroll         103      586
Shakespeare     451      196


In [192]:
(451+586)/(103+586+451+196)

0.7761976047904192

Well, the model is designed to predict Austen/Carroll, but it does fairly well predicting Carroll/Not Carroll (Shakespeare)... From the crosstab it looks better than from the score, which is strange. Maybe it is labeling all the "Shakespeare" as incorrect because they don't have the Austen label? Looks like that is the case. A quick calculation says that it's actually 78% accurate at Carroll/Not Carroll.

In [188]:
# So I don't have to change all the previous code...
df = df_ham_per.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

y_pred = lr.predict(X_test)
print(lr.score(X_test, y_test))
print(pd.crosstab(y_test, y_pred))

0.669172932331
col_0        Austen  Carroll
text_source                 
Austen         1424       48
Shakespeare     453      203


In [191]:
(203 + 1424)/(203+1424+453+48)

0.7645676691729323

Same issue here -- it counts all the Shakespeare labels as incorrect. In terms of Austen/Not Austen, it's actually 76% accurate.

In [189]:
# So I don't have to change all the previous code...
df = df_par_ali.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

y_pred = lr.predict(X_test)
print(lr.score(X_test, y_test))
print(pd.crosstab(y_test, y_pred))

0.438622754491
col_0        Austen  Carroll
text_source                 
Carroll         103      586
Edgewood        442      205


In [193]:
(442+586)/(103+442+586+205)

0.7694610778443114

For Edgewood vs. Carroll, it's 77% accurate on Carroll/Not Carroll, even though the score doesn't reflect that.

In [190]:
# So I don't have to change all the previous code...
df = df_par_per.copy()

Y = df['text_source']
X = np.array(df.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

y_pred = lr.predict(X_test)
print(lr.score(X_test, y_test))
print(pd.crosstab(y_test, y_pred))

0.646616541353
col_0        Austen  Carroll
text_source                 
Austen         1376       96
Edgewood        395      261


In [194]:
(1376+261)/(1376+96+395+261)

0.769266917293233

Also 77% accurate for Austen/Not Austen when using Edgewood.